In [ ]:
import numpy as np
import forwardModel as fw
import matplotlib.pyplot as plt
from myEnKF import myEnKF

In [ ]:
# We define the control parameters here 
rayleigh = 45 # pick the ones you want
prandtl = 10.
b = 8./3.

In [ ]:
#integration time parameter
dt = 1.e-3      # This is time step size
T = 15.         # Total integration time, can be as short as 10 to speed things up
n_steps = int( np.ceil( T / dt)  )
time = np.linspace(0., T, n_steps + 1, endpoint=True) # array of discrete times

In [ ]:
#initial condition for the true reference trajectory
x0 = np.array( [0., 1., 2.], dtype=float );

In [ ]:
#numerical integration given initial conditions and control parameters
xt = fw.forwardModel_r( x0, time, rayleigh, prandtl, b) 

In [ ]:
## How often do we observe the true state? 
dtobs = 1. # time between observations
# Which variables do we observe? 
WhichVariablesAreObserved = np.array( [1, 1, 1], dtype=float )
                    #  Determines which variables are available to
                    #  the EnKF. For example:
                    #  WhichVariablesAreObserved = [1 1 1]; 
                    #  means: X, Y, Z are observed
                    #  WhichVariablesAreObserved = [1 0 1];
                    #  means: X and Z are observed
                    #  WhichVariablesAreObserved = [1 0 0];
                    # means: X is observed
sigobs = 4.  # standard deviation of the observation noise
# We generate the synthetic data 
#  Construct observation matrix H
#  ........................................................................
H = np.diag( WhichVariablesAreObserved )

In [ ]:
y_size = int( np.sum( WhichVariablesAreObserved ) )
H = np.zeros( (y_size, 3), dtype=float)
iy = 0
for ix in range(3):
    if WhichVariablesAreObserved[ix] > 0:
        H[iy,ix] = 1.
        iy = iy + 1

In [ ]:
nobs = int( np.ceil( T / dtobs ) ) - 1  # number of times observations are performed
                                        # no observation at t=0 
gap  = int( dtobs/dt )# number of time steps between each observation 
time_obs = time[gap::gap]
# Generate vector of observations
y = np.zeros( (y_size, nobs), dtype=float)
R = np.diag( np.tile(sigobs**2, y_size) )
sqrt_s = np.sqrt(R)
# y = Hxt 
y = np.dot( H, xt[:,gap::gap] )
# compute observation error 
noise = np.dot(sqrt_s , np.random.normal( loc=0., scale=1.0, size=np.shape(y) ) )
# y = Hxt + epsilon 
y = y + noise

In [ ]:
#
# Plot reference trajectory and the observations that will be fed to the EnKF 
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(10,7), sharex=True)
iobs = 0
for k, comp in enumerate (["X","Y","Z"]):
    ax[k].plot(time, xt[k,:])
    if WhichVariablesAreObserved[k] > 0:
        ax[k].errorbar(time_obs, y[iobs,:], yerr=sqrt_s[iobs,iobs], fmt='o', markersize=3, capsize=4, label='obs')
        ax[k].legend(bbox_to_anchor=(1.01, 1),loc='upper left', frameon=True)
        iobs = iobs + 1
    ax[k].set_ylabel(comp)
ax[-1].set_xlabel('Time')
ax[-1].set_xlim(time[0],time[-1])
#plt.show()
plt.close()

In [ ]:
# Now let us run the Ensemble Kalman Filter
# Initialization of the Ensemble
Ne = 50                  # Number of ensemble members  
x0ens = np.array( [0.,0.,0.], dtype=float ) # The ensemble is centered on a zero initial condition
sigens = 10          # standard deviation of the ensemble
# initial covariance matrix
P0 =(sigens**2)*np.eye(3, dtype=float)
#Now run the EnKF 
xEnKF, x_ens = myEnKF( n_steps, dt, nobs, time_obs, gap, Ne, H, R, y, x0ens, P0, rayleigh, prandtl, b)

In [ ]:
# Compute Euclidean error wrt true state which is known in this synthetic game
# cumulative error
cum_error_comp = np.sqrt( np.sum( (xt - xEnKF)**2, axis=1) ) 
cum_error      = np.sqrt( np.sum( cum_error_comp**2) )
norm_comp = np.sqrt( np.sum( xt**2, axis=1) ) 
norm      = np.sqrt( np.sum( norm_comp**2) )
print()
print(" Relative errors in %")
for k, comp in enumerate (["X","Y","Z"]):
    print("  "+(comp+"-component: %4.1f  ") % ( 100.*cum_error_comp[k]/norm_comp[k]) )
print()
print(("  3-component: %4.1f  ") % ( 100.*cum_error / norm  ) )

In [ ]:
#Plot results
plot_ensemble = True # Change to True if you want to visualize ensemble members
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(10,7), sharex=True)
iobs = 0
for k, comp in enumerate (["X","Y","Z"]):
    ax[k].plot(time, xt[k,:], label='true')
    ax[k].plot(time, xEnKF[k,:], label='EnKF')
    if plot_ensemble is True:
        for e in range(Ne):
            ax[k].plot(time, x_ens[k,:,e], c='k', lw=0.1, zorder=-4)
    if WhichVariablesAreObserved[k] > 0:
        ax[k].errorbar(time_obs, y[iobs,:], yerr=sqrt_s[iobs,iobs], fmt='o', markersize=3, capsize=4, label='obs')
        iobs = iobs + 1
    ax[k].legend(bbox_to_anchor=(1.01, 1),loc='upper left', frameon=True)
    ax[k].set_ylabel(comp)
    ax[k].title.set_text( ("Error for "+comp+"-component: %4.1f %% ") % ( 100.*cum_error_comp[k]/norm_comp[k])  )
ax[-1].set_xlabel('Time')
ax[-1].set_xlim(time[0],time[-1])
plt.show()

Now consider the following questions: 
Q1: All other parameters remaining constant, find the maximum value of sigma_obs which leads to an acceptable behaviour of the EnKF (define what you would consider an acceptable behaviour of the EnKF).
Q2: Now with all other parameters remaining constant (σobs being equal to the value you just found),findthelargest
value of dtobs whichleads to an acceptable behaviour of the EnKF.
Q3: Is there a connection between this value and the typical time scale of the dynamics of the model?
Q4: Set dt_obs to half the maximum value you just found. 
Keeping all the other parameter constant, assume now that only X or Y is observed. 
Comment on the quality of the EnKF estimate based on either option.
Q5: It may be that the quality of the results depend strongly on the variable which is observed
(X or Y). Would you have an explanation for this, based on a simple analysis of the equations 
that govern the dynamics of the L63 model?
Q6: Q5: All other parameters remaining constant, find the minimum number of elements of the ensemble which leads to a good behaviour of the EnKF.

For those done, consider the following question: 
Is the scatter of the ensemble a good proxy for the error? 
(keep in mind that in a real situation, we do not know what the truth is, so we need
proxies to estimate how good the EnKF is doing